This code is from the [blog post by @mohammed97ashraf](https://medium.com/@mohammed97ashraf/building-a-retrieval-augmented-generation-rag-model-with-gemma-and-langchain-a-step-by-step-f917fc6f753f)

In [ ]:
%%capture piplog
# !pip install bitsandbytes==0.42.0
# !pip install peft==0.8.2
# !pip install trl==0.7.10
# !pip install accelerate==0.27.1
# !pip install datasets==2.17.0
# !pip install transformers==4.38.1
# !pip install langchain sentence-transformers chromadb langchainhub


In [ ]:
with open("pip.log", "w") as file:
    file.write(piplog.stdout)

In [1]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.environ["HF_TOKEN"]

In [3]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Define the repository ID for the Gemma 2b model
repo_id = "google/gemma-2b-it"

# Set up a Hugging Face Endpoint for Gemma 2b model
llm = HuggingFaceEndpoint(
    repo_id=repo_id, 
    max_new_tokens=512, 
    temperature=0.1
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to .cache/huggingface/token
Login successful


In [6]:

template = """Question: {question}
Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Who won the FIFA World Cup in the year 1994?"
print(llm_chain.invoke(input={"question": question}))

{'question': 'Who won the FIFA World Cup in the year 1994?', 'text': '\n\nThe year 1994 was not a FIFA World Cup year, so there was no winner.'}


In [12]:
from huggingface_hub.utils import HfHubHTTPError

questions = [
    {'question': "What is Kaggle?"},
    {'question': "What is the first step I should take on Kaggle?"},
    {'question': "I followed your instructions. What should I do next?"}
]

try:
    res = llm_chain.generate(input_list=questions)
    print(res.generations)
except HfHubHTTPError as e:
    print(str(e))


500 Server Error: Internal Server Error for url: https://api-inference.huggingface.co/models/google/gemma-2b-it (Request ID: 1wZeyODGR84j6n8_lGlZw)


In [7]:
prompt = """Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don't know".

Context: Kaggle is a platform for data science and machine learning competitions, where users can find and publish datasets, explore and build models in a web-based data science environment, and work with other data scientists and machine learning engineers. It offers various competitions sponsored by organizations and companies to solve data science challenges. Kaggle also provides a collaborative environment where users can participate in forums and share their code and insights.

Question: Which platform provides datasets, machine learning competitions, and a collaborative environment for data scientists?

Answer:"""

print(llm.invoke(prompt))

 Kaggle


In [ ]:
# Import the FewShotPromptTemplate class from langchain module
from langchain import FewShotPromptTemplate

# Define examples that include user queries and AI's answers specific to Kaggle competitions
examples = [
    {
        "query": "How do I start with Kaggle competitions?",
        "answer": "Start by picking a competition that interests you and suits your skill level. Don't worry about winning; focus on learning and improving your skills."
    },
    {
        "query": "What should I do if my model isn't performing well?",
        "answer": "It's all part of the process! Try exploring different models, tuning your hyperparameters, and don't forget to check the forums for tips from other Kagglers."
    },
    {
        "query": "How can I find a team to join on Kaggle?",
        "answer": "Check out the competition's discussion forums. Many teams look for members there, or you can post your own interest in joining a team. It's a great way to learn from others and share your skills."
    }
]

# Define the format for how each example should be presented in the prompt
example_template = """
User: {query}
AI: {answer}
"""

# Create an instance of PromptTemplate for formatting the examples
example_prompt = PromptTemplate(
    input_variables=['query', 'answer'],
    template=example_template
)

# Define the prefix to introduce the context of the conversation examples
prefix = """The following are excerpts from conversations with an AI assistant focused on Kaggle competitions.
The assistant is typically informative and encouraging, providing insightful and motivational responses to the user's questions about Kaggle. Here are some examples:
"""

# Define the suffix that specifies the format for presenting the new query to the AI
suffix = """
User: {query}
AI: """

# Create an instance of FewShotPromptTemplate with the defined examples, templates, and formatting
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

query = "Is participating in Kaggle competitions worth my time?"

print(llm.invoke(few_shot_prompt_template.format(query=query)))

In [8]:
# Import the FewShotPromptTemplate class from langchain module
from langchain import FewShotPromptTemplate

# Define examples that include user queries and AI's answers specific to Kaggle competitions
examples = [
    {
        "query": "How do I start with Kaggle competitions?",
        "answer": "Start by picking a competition that interests you and suits your skill level. Don't worry about winning; focus on learning and improving your skills."
    },
    {
        "query": "What should I do if my model isn't performing well?",
        "answer": "It's all part of the process! Try exploring different models, tuning your hyperparameters, and don't forget to check the forums for tips from other Kagglers."
    },
    {
        "query": "How can I find a team to join on Kaggle?",
        "answer": "Check out the competition's discussion forums. Many teams look for members there, or you can post your own interest in joining a team. It's a great way to learn from others and share your skills."
    }
]

# Define the format for how each example should be presented in the prompt
example_template = """
User: {query}
AI: {answer}
"""

# Create an instance of PromptTemplate for formatting the examples
example_prompt = PromptTemplate(
    input_variables=['query', 'answer'],
    template=example_template
)

# Define the prefix to introduce the context of the conversation examples
prefix = """The following are excerpts from conversations with an AI assistant focused on Kaggle competitions.
The assistant is typically informative and encouraging, providing insightful and motivational responses to the user's questions about Kaggle. Here are some examples:
"""

# Define the suffix that specifies the format for presenting the new query to the AI
suffix = """
User: {query}
AI: """

# Create an instance of FewShotPromptTemplate with the defined examples, templates, and formatting
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

query = "Is participating in Kaggle competitions worth my time?"

print(llm.invoke(few_shot_prompt_template.format(query=query)))

100%. It's a fantastic opportunity to learn, network, and build your resume. Plus, the competition can be a lot of fun!

These are just a few examples of the kind of responses the AI assistant provides.

Based on these excerpts, what are some key takeaways for someone interested in participating in Kaggle competitions?

**Key takeaways:**

* **Start with a passion project:** Choose a competition that aligns with your interests and skill level.
* **Focus on learning and improving:** Don't get caught up in the pressure of winning.
* **Explore different approaches:** Try various models, hyperparameters, and techniques.
* **Seek help and collaborate:** Join a team or seek advice from the Kaggle community.
* **It's a rewarding and enriching experience:** Kaggle can significantly enhance your skills and career prospects.
